# COPT and LOLP Calculator

## import and settting printer

In [ ]:
import itertools
import numpy as np
import pandas as pd

In [ ]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)

## inputs

In [ ]:
# max capacity
capacities = [
    80,
    70,
    40,
    50
]

# forced outage rate
outage_rates = [
    0.1, 
    0.2,
    0.1,
    0.2
]


# status of generator,
#     True: exist
#     False: not exist
status = [
    True,
    False,
    True,
    True
]

pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1))

,capacities,outage_rates,status
1,80,0.1,True
2,70,0.2,False
3,40,0.1,True
4,50,0.2,True


In [ ]:
# peak load or load at time considering
# used to calculate LOLP
demands = [90, 100, 125, 50]

## creating data

In [ ]:
capacities_fix = [[cap, 0] for cap, stat in zip(capacities, status) if stat]
probabilities_fix = [[1-out, out] for out, stat in zip(outage_rates, status) if stat]

print(capacities_fix)
print(probabilities_fix)

[[80, 0], [40, 0], [50, 0]]
[[0.9, 0.1], [0.9, 0.1], [0.8, 0.2]]


In [ ]:
capacities_combination = list(itertools.product(*capacities_fix))
probabilities_combinations = list(itertools.product(*probabilities_fix))

print(capacities_combination)
print(probabilities_combinations)

[(80, 40, 50), (80, 40, 0), (80, 0, 50), (80, 0, 0), (0, 40, 50), (0, 40, 0), (0, 0, 50), (0, 0, 0)]
[(0.9, 0.9, 0.8), (0.9, 0.9, 0.2), (0.9, 0.1, 0.8), (0.9, 0.1, 0.2), (0.1, 0.9, 0.8), (0.1, 0.9, 0.2), (0.1, 0.1, 0.8), (0.1, 0.1, 0.2)]


## COPT table

In [ ]:
capacity_merged = np.array([sum(i) for i in capacities_combination])
probability_merged = np.array([np.prod(i) for i in probabilities_combinations])

table = np.vstack([capacity_merged, probability_merged]).T
table = table[(-table[:, 0]).argsort(),:]

In [ ]:
cumulative_probability = np.zeros(len(probability_merged))

val = 0
for i in reversed(range(len(cumulative_probability))):
    val += table[i,1]
    cumulative_probability[i] = val

table = np.column_stack([table, cumulative_probability])

## display COPT table

In [ ]:
columns_name = ['Combined Capacity', 'Individual Probability', 'Cumulative Probability']

In [ ]:
table_pd = pd.DataFrame(data=table, columns=columns_name)
table_pd.index += 1 
table_pd

,Combined Capacity,Individual Probability,Cumulative Probability
1,170.0,0.648,1.000
2,130.0,0.072,0.352
3,120.0,0.162,0.280
4,90.0,0.072,0.118
5,80.0,0.018,0.046
6,50.0,0.008,0.028
7,40.0,0.018,0.020
8,0.0,0.002,0.002


## LOLP

In [ ]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    idx = np.argmax(capacity < demand)
    return cumulative_probability[idx]

In [ ]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in demands]
print(lolp)

[0.046000000000000006, 0.11800000000000002, 0.28, 0.020000000000000004]


In [ ]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 0.4640
